# Momentum 策略

本專案旨在系統性地重建並驗證美股市場中動能投資組合 (Momentum Portfolios) 的表現與穩健性。透過使用 CRSP 股票資料、Fama-French 因子檔以及外部提供的 decile 報酬率檔 (DM 與 KRF)，本專案執行以下主要步驟：

1. 資料前處理與清理

    * 對 CRSP 股票資料進行篩選（只留普通股與主要交易所股票），合併正常報酬與退市報酬，確保資料正確性。
    
    * 計算 lag 市值，避免 look-ahead bias。

3. 動能因子計算

    * 以 12-2 月累積log報酬作為 Ranking Return
    * 根據全市場 (DM) 與 NYSE-only (KRF) 分組基準，將股票分成 10 個 decile。

5. 投資組合報酬計算

    * 根據 lagged 市值加權，計算每個 decile 的每月報酬率。
    
    * 計算動能投資組合 (WML: Winner-Minus-Loser) 報酬

6. 績效與穩健性驗證

    * 與 Daniel & Moskowitz、Kenneth R. French 官方 decile 報酬率檔案進行相關性檢驗。
    
    * 輸出 decile & WML 投資組合的年化報酬、波動度、Sharpe Ratio、偏態與與官方資料的相關性。

7. 視覺化與結論

    * 繪製 1929-2024 & 2010-2024 年間的累積 log 報酬率圖，直觀比較動能策略在不同時期的表現。
    
    ** 提供研究洞察與未來可能的市場 regime shift 影響。 **

In [3]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Functions
from data_process import clean_CRSP_Stocks, prepare_momentum_data, clean_FF_mkt
from portfolio import assign_momentum_deciles, calculate_portfolio_returns, get_wml
from evaluation import clean_DM_ret, clean_KRF_ret, generate_dm_summary_stats, generate_krf_summary_stats

In [4]:
if __name__ == "__main__":
    sys.stdout = open("../output/log.txt", "w", encoding="utf-8")
    # Data Cleaning & Integration
    CRSP_Stocks = pd.read_pickle('../data/raw/CRSP.pkl')
    CRSP_Stocks = CRSP_Stocks.rename({"permno":"PERMNO", "exchcd":"EXCHCD", "ret":"RET", "shrcd":"SHRCD", "shrout":"SHROUT", "prc":"PRC", "dlret":"DLRET"}, axis=1)
    CRSP_Stocks = CRSP_Stocks.loc[:, ["date", "PERMNO", "EXCHCD", "RET", "SHRCD", "SHROUT", "PRC", "DLRET"]]
    CRSP_Stocks_Momentum = prepare_momentum_data(CRSP_Stocks)
    print("Step 1: Loaded and cleaned CRSP stock data")
    print(CRSP_Stocks_Momentum)
    print("\n" * 3)
    
    # Momentum Deciles
    CRSP_Stocks_Momentum_decile = assign_momentum_deciles(CRSP_Stocks_Momentum)
    print("Step 2: Constructed Momentum factor and assigned decile ranks")
    print(CRSP_Stocks_Momentum_decile)
    print("\n" * 3)

    # Portfolio Return Construction
    FF_mkt = pd.read_csv("../data/raw/F-F_Research_Data_Factors.CSV", skiprows=3, nrows=1182)
    FF_mkt = clean_FF_mkt(FF_mkt)
    print("Input: Fama-French Market DataFrame (FF_mkt)")
    print(FF_mkt)
    print("")
    
    CRSP_Stocks_Momentum_returns = calculate_portfolio_returns(CRSP_Stocks_Momentum_decile, FF_mkt)
    print("Step 3: Computed value-weighted monthly portfolio returns")
    print(CRSP_Stocks_Momentum_returns)
    print("\n" * 3)

    # DM Model Backtest Validation
    DM_returns = pd.read_csv('../data/raw/m_m_pt_tot.txt', sep="\\s+", header=None)
    DM_returns = clean_DM_ret(DM_returns)
    print("Input: Momentum portfolio returns from Daniel\'s website (DM_returns)")
    print(DM_returns)
    print("")

    print("Step 4: Validated DM portfolio performance against benchmark data")
    DM_res = generate_dm_summary_stats(CRSP_Stocks_Momentum_returns, DM_returns)
    print(DM_res)
    print("\n" * 3)

    # KRF Model Backtest Validation
    KRF_returns = pd.read_csv("10_Portfolios_Prior_12_2.csv", skiprows=10, nrows=1176, index_col=0)
    KRF_returns = clean_KRF_ret(KRF_returns)
    print("Input: Momentum portfolio returns from French\'s website (KRF_returns)")
    print(KRF_returns)
    print("")

    print("Step 5: Validated KRF portfolio performance against benchmark data")
    KRF_res = generate_krf_summary_stats(CRSP_Stocks_Momentum_returns, KRF_returns)
    print(KRF_res)
    print("\n" * 3)

    # Strategy Performance Visualization
    wml_port_rets = get_wml(CRSP_Stocks_Momentum_returns, "DM_Ret").merge(get_wml(CRSP_Stocks_Momentum_returns, "KRF_Ret"), on=["Year", "Month", "decile"])
    wml_port_rets = wml_port_rets[wml_port_rets.decile=="WML"].drop(["decile"], axis=1)
    wml_port_rets.loc[:, "str_date"] = wml_port_rets["Year"].astype(str) + wml_port_rets["Month"].astype(str).str.zfill(2)
    wml_port_rets.loc[:, "date"] = pd.to_datetime(wml_port_rets.loc[:, "str_date"], format='%Y%m')
    wml_port_rets = wml_port_rets.set_index("date")
    wml_port_rets.drop(["Year", "Month", "str_date"], axis=1, inplace=True)
    wml_port_rets = np.log(wml_port_rets+1).cumsum()
    wml_port_rets.columns = ["DM", "KRF"]
    wml_port_rets.plot()
    plt.title("Cumulative Log Returns of MOM portfolios 1929-2024")
    plt.show()

    wml_port_rets_2010 = wml_port_rets.loc["2010-01-01":]
    wml_port_rets_2010 = wml_port_rets_2010 - wml_port_rets_2010.loc["2010-01-01", :]
    wml_port_rets_2010.plot()
    plt.title("Cumulative Log Returns of MOM portfolios 2010-2024")
    plt.show()

    sys.stdout.close()
    sys.stdout = sys.__stdout__

FileNotFoundError: [Errno 2] No such file or directory: '10_Portfolios_Prior_12_2.csv'